In [ ]:
#| default_exp estacoes
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path


In [ ]:

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Anatel

> Este módulo consolida as bases da Anatel e realiza pós-processamento dos dados obtidos.

In [ ]:
# | export
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from fastcore.foundation import L
from fastcore.parallel import parallel

from extracao.constants import COLS_SRD
from extracao.location import Geography

from extracao.datasources.aeronautica import Aero
from extracao.datasources.base import Base
from extracao.datasources.mosaico import MONGO_URI
from extracao.datasources.sitarweb import SQLSERVER_PARAMS, Radcom, Stel
from extracao.datasources.smp import Smp
from extracao.datasources.srd import SRD
from extracao.datasources.telecom import Telecom
from extracao.format import LIMIT_FREQ, merge_on_frequency

In [ ]:
# | export
load_dotenv(find_dotenv(), override=True)
pd.options.mode.copy_on_write = True

## Base Consolidada ANATEL

In [ ]:
# |export
class Estacoes(Base):
	"""Helper Class to aggregate and process the data from different sources"""

	def __init__(
		self,
		sql_params: dict = SQLSERVER_PARAMS,
		mongo_uri: str = MONGO_URI,
		limit: int = 0,
		parallel: bool = True,
		read_cache: bool = True,
		reprocess_sources: bool = False,
	):
		self.sql_params = sql_params
		self.mongo_uri = mongo_uri
		self.limit = limit
		self.parallel = parallel
		self.read_cache = read_cache
		self.reprocess_sources = reprocess_sources
		self.init_data_sources()

	@property
	def columns(self):
		return COLS_SRD

	@property
	def stem(self):
		return 'estacoes'

	@staticmethod
	def _update_source(class_instance):
		"""Helper functions to update and save the individual data sources"""
		try:
			class_instance.update()
			class_instance.save()
		except Exception as e:
			print(f'Erro ao atualizar a classe {class_instance.__class__.__name__}: {e}')
		return class_instance

	def init_data_sources(self):
		"""Initializes the individual classes and saves them in a property list"""
		self.sources = L(
			[
				Telecom(self.mongo_uri, self.limit, self.read_cache),
				Smp(self.mongo_uri, self.limit, self.read_cache),
				SRD(self.mongo_uri, self.limit, self.read_cache),
				Stel(self.sql_params, self.read_cache),
				Radcom(self.sql_params, self.read_cache),
				Aero(self.read_cache),
			]
		)

	def extraction(self) -> L:
		if not self.reprocess_sources and self.read_cache:
			return self.sources.attrgot('df')
		if self.parallel:
			self.sources = parallel(
				Estacoes._update_source,
				self.sources,
				n_workers=len(self.sources),
				progress=False,
			)
		else:
			self.sources = self.sources.map(Estacoes._update_source)
		return self.sources.attrgot('df')

	def update(self):
		df = self.extraction()
		self.df = self._format(df)

	@staticmethod
	def _simplify_sources(df):
		df['Fonte'] = df['Fonte'].str.replace(
			'ICAO-CANALIZACAO-VOR/ILS/DME | AISWEB-CANALIZACAO-VOR/ILS/DME',
			'CANALIZACAO-VOR/ILS/DME',
		)
		df['Fonte'] = df['Fonte'].str.replace(
			r'(ICAO-)?(AISWEB-)?CANALIZACAO-VOR/ILS/DME',
			'CANALIZACAO-VOR/ILS/DME',
			regex=True,
		)

		return df

	@staticmethod
	def _remove_invalid_frequencies(df):
		df['Frequência'] = df['Frequência'].astype('float')
		df.sort_values(['Frequência', 'Latitude', 'Longitude'], ignore_index=True, inplace=True)
		row_filter = df['Frequência'] > LIMIT_FREQ
		Base.register_log(
			df,
			f'Frequência removida. Valor superior a {LIMIT_FREQ}MHz.',
			'Frequência',
			row_filter,
		)
		return df[~row_filter].reset_index(drop=True)

	def _format(
		self,
		dfs: L,  # List with the individual API sources
	) -> pd.DataFrame:  # Processed DataFrame
		aero = dfs.pop()
		anatel = pd.concat(dfs, ignore_index=True, copy=False).astype('string', copy=False)
		df = merge_on_frequency(anatel, aero)
		df = Geography(df).validate()
		df = Estacoes._simplify_sources(df)
		df = Estacoes._remove_invalid_frequencies(df)
		df = df.astype('string', copy=False).replace('-1.0', '-1')  # .fillna('-1')
		df = df.astype('category', copy=False)
		return df
		# return df.loc[:, self.columns]


In [ ]:
data = Estacoes()

In [ ]:
dfs = data.extraction()

In [ ]:
aero = dfs.pop()
anatel = pd.concat(dfs, ignore_index=True, copy=False).astype('string', copy=False)
df = merge_on_frequency(anatel, aero)

/tmp/ipykernel_1916517/1210101140.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  anatel = pd.concat(dfs, ignore_index=True, copy=False).astype('string', copy=False)


Logging: Calculando distância entre estações mescladas.

  0%|          | 0/55928 [00:00<?, ?it/s]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023110 entries, 0 to 1023109
Data columns (total 29 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   Frequência                   1023110 non-null  string
 1   Entidade                     1006071 non-null  string
 2   Fistel                       1021523 non-null  string
 3   Serviço                      1021523 non-null  string
 4   Estação                      1017709 non-null  string
 5   Latitude                     1005771 non-null  string
 6   Longitude                    1005771 non-null  string
 7   Código_Município             1021523 non-null  string
 8   Município                    533869 non-null   string
 9   UF                           1020381 non-null  string
 10  Classe                       1021078 non-null  string
 11  Classe_Emissão               1016081 non-null  string
 12  Largura_Emissão(kHz)         1021102 non-null  string
 1

In [ ]:
df.isna().sum()

Frequência                           0
Entidade                         17039
Fistel                            1587
Serviço                           1587
Estação                           5401
Latitude                         17339
Longitude                        17339
Código_Município                  1587
Município                       489241
UF                                2729
Classe                            2032
Classe_Emissão                    7029
Largura_Emissão(kHz)              2008
Validade_RF                       9240
Status                            1587
Fonte                                0
Multiplicidade                       0
Log                               1587
Cota_Base_Torre(m)             1010062
Potência_Transmissor(W)         995562
Ganho_Antena(dBd)               996915
Ângulo_Elevação_Antena          996564
Azimute_Antena                  995930
Altura_Antena(m)                995144
Atenuação_Linha(db/100m)        996751
Perdas_Acessórias_Linha(d

In [ ]:
from extracao.location import Geography

In [ ]:
g = Geography(df)

Logging: Coordenadas nulas.

  0%|          | 0/17339 [00:00<?, ?it/s]

Logging: Código do Município nulo.

  0%|          | 0/1587 [00:00<?, ?it/s]

Logging: Código do Município não numérico.

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
g.df.isna().sum()

Frequência                           0
Entidade                         17039
Fistel                            1587
Serviço                           1587
Estação                           5401
Latitude                         17339
Longitude                        17339
Código_Município                  1589
Município                       489241
UF                                2729
Classe                            2032
Classe_Emissão                    7029
Largura_Emissão(kHz)              2008
Validade_RF                       9240
Status                            1587
Fonte                                0
Multiplicidade                       0
Log                                  0
Cota_Base_Torre(m)             1010062
Potência_Transmissor(W)         995562
Ganho_Antena(dBd)               996915
Ângulo_Elevação_Antena          996564
Azimute_Antena                  995930
Altura_Antena(m)                995144
Atenuação_Linha(db/100m)        996751
Perdas_Acessórias_Linha(d

In [ ]:
c = g.df['Código_Município'].isna() & df['Código_Município'].notna()
df[c]

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
434235,18057.5,TELEFONICA BRASIL S.A.,50417179405,019,1015168474,-23.5222,-46.917,JANDIRA,3525003,SP,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
529246,19617.5,TELEFONICA BRASIL S.A.,50417179405,019,1015168466,-23.5136,-46.9295,ITAPEVI,3522505,SP,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
g.df[c].loc[434235, 'Log']

'[{"Processamento": "Registro expandido nos componentes individuais Largura_Emissão(kHz) e Classe_Emissão", "Coluna": "Designação_Emissão", "Original": "55M0D7W"}, {"Processamento": "Código do Município não numérico.", "Coluna": "#Código_Município", "Original": "JANDIRA"}]'

In [ ]:
g.merge_df_with_ibge()

Logging: Código do Município não consta no IBGE.
Município normalizado e UF serão usados como chave para validação.

  0%|          | 0/87 [00:00<?, ?it/s]

Logging: Valor da coluna Município substituído conforme consta no IBGE

  0%|          | 0/39 [00:00<?, ?it/s]

Logging: Valor da coluna Código_Município substituído conforme consta no IBGE

  0%|          | 0/39 [00:00<?, ?it/s]

In [ ]:
g.df.isna().sum()

Frequência                           0
Entidade                         17039
Fistel                            1587
Serviço                           1587
Estação                           5401
Latitude                         17339
Longitude                        17339
Código_Município                  1592
Município                       489244
UF                                2729
Classe                            2032
Classe_Emissão                    7029
Largura_Emissão(kHz)              2008
Validade_RF                       9240
Status                            1587
Fonte                                0
Multiplicidade                       0
Log                                  0
Cota_Base_Torre(m)             1010062
Potência_Transmissor(W)         995562
Ganho_Antena(dBd)               996915
Ângulo_Elevação_Antena          996564
Azimute_Antena                  995930
Altura_Antena(m)                995144
Atenuação_Linha(db/100m)        996751
Perdas_Acessórias_Linha(d

In [ ]:
g.normalize_location_names()

Logging: Coluna Município normalizada como consta no IBGE.

  0%|          | 0/734167 [00:00<?, ?it/s]

Logging: Coluna UF normalizada como consta no IBGE.

  0%|          | 0/1161 [00:00<?, ?it/s]

In [ ]:
g.fill_missing_coords()

Logging: Coordenadas ausentes. Latitude do Município inserida.

  0%|          | 0/17339 [00:00<?, ?it/s]

Logging: Coordenadas ausentes. Longitude do Município inserida.

  0%|          | 0/17339 [00:00<?, ?it/s]

In [ ]:
g.df.isna().sum()

Frequência                           0
Entidade                         17039
Fistel                            1587
Serviço                           1587
Estação                           5401
Latitude                             0
Longitude                            0
Código_Município                  1592
Município                         1590
UF                                1601
Classe                            2032
Classe_Emissão                    7029
Largura_Emissão(kHz)              2008
Validade_RF                       9240
Status                            1587
Fonte                                0
Multiplicidade                       0
Log                                  0
Cota_Base_Torre(m)             1010062
Potência_Transmissor(W)         995562
Ganho_Antena(dBd)               996915
Ângulo_Elevação_Antena          996564
Azimute_Antena                  995930
Altura_Antena(m)                995144
Atenuação_Linha(db/100m)        996751
Perdas_Acessórias_Linha(d

In [ ]:
g.intersect_coordinates_on_poligon()

/home/ronaldo/code/rfdatahub/extracao/location.py:283: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_joined['LAT'] = gdf_joined.geometry.centroid.y.astype('string', copy=False)
/home/ronaldo/code/rfdatahub/extracao/location.py:284: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_joined['LON'] = gdf_joined.geometry.centroid.x.astype('string', copy=False)


In [ ]:
g.fill_missing_city_info()

Logging: Código_Município Ausente. Informação resgatada à partir da intersecção das coordenadas no polígono 
territorial.

  0%|          | 0/1431 [00:00<?, ?it/s]

Logging: Município Ausente. Informação resgatada à partir da intersecção das coordenadas no polígono territorial.

  0%|          | 0/1431 [00:00<?, ?it/s]

Logging: UF Ausente. Informação resgatada à partir da intersecção das coordenadas no polígono territorial.

  0%|          | 0/1426 [00:00<?, ?it/s]

In [ ]:
g.log['filled_city_info'].sum()

1431

In [ ]:
g.df.isna().sum()

Frequência                           0
Entidade                         17039
Fistel                            1587
Serviço                           1587
Estação                           5401
Latitude                             0
Longitude                            0
Código_Município                   161
Município                          161
UF                                 175
Classe                            2032
Classe_Emissão                    7029
Largura_Emissão(kHz)              2008
Validade_RF                       9240
Status                            1587
Fonte                                0
Multiplicidade                       0
Log                                  0
Cota_Base_Torre(m)             1010064
Potência_Transmissor(W)         995564
Ganho_Antena(dBd)               996917
Ângulo_Elevação_Antena          996566
Azimute_Antena                  995932
Altura_Antena(m)                995146
Atenuação_Linha(db/100m)        996753
Perdas_Acessórias_Linha(d

In [ ]:
g.substitute_divergent_coordinates()

/home/ronaldo/code/rfdatahub/extracao/location.py:331: FutureWarning: Operation between non boolean Series with different indexes will no longer return a boolean result in a future version. Cast both Series to object type to maintain the prior behavior.
  wrong_city_coords &= self.log['city_normalized']


Logging: Latitude divergente. Valor substituído pelo centróide da intersecção das coordenadas no polígono 
territorial.

  0%|          | 0/32470 [00:00<?, ?it/s]

Logging: Longitude divergente. Valor substituído pelo centróide da intersecção das coordenadas no polígono 
territorial.

  0%|          | 0/32470 [00:00<?, ?it/s]

In [ ]:
a = g.df['Código_Município'] != g.df['CD_MUN']
g.df[a]

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Relatório_Canal,Município_IBGE,Latitude_IBGE,Longitude_IBGE,UF_IBGE,CD_MUN,NM_MUN,SIGLA_UF,LAT,LON
64,0.048,COMPANHIA DE GERACAO TERMICA DE ENERGIA ELETRICA,50420217282,019,641233027,-29.356,-50.8119,4304408,Canela,RS,...,<NA>,Canela,-29.356,-50.8119,RS,4318200,São Francisco de Paula,RS,-29.275,-50.799722
120,0.064,COMPANHIA DE GERACAO TERMICA DE ENERGIA ELETRICA,50420217282,019,641233027,-29.356,-50.8119,4304408,Canela,RS,...,<NA>,Canela,-29.356,-50.8119,RS,4318200,São Francisco de Paula,RS,-29.275,-50.799722
121,0.064,FURNAS CENTRAIS ELETRICAS S A,01030052263,019,683388762,-24.2058,-50.9185,4117305,Ortigueira,PR,...,<NA>,Ortigueira,-24.2058,-50.9185,PR,4107603,Faxinal,PR,-24.052778,-51.155278
162,0.072,FURNAS CENTRAIS ELETRICAS S A,01030052263,019,683388762,-24.2058,-50.9185,4117305,Ortigueira,PR,...,<NA>,Ortigueira,-24.2058,-50.9185,PR,4107603,Faxinal,PR,-24.052778,-51.155278
196,0.08,COMPANHIA DE GERACAO TERMICA DE ENERGIA ELETRICA,50420217282,019,641233434,-27.4242,-51.7668,4213104,Piratuba,SC,...,<NA>,Piratuba,-27.4242,-51.7668,SC,4312203,Maximiliano de Almeida,RS,-27.523611,-51.784722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020632,128.45,"COMANDO DA AERONAUTICA | ACC-U C-261/450, MORR...",50415723132,507,1007691252,-28.0157,-49.5925,4218905,Urubici,SC,...,<NA>,Urubici,-28.0157,-49.5925,SC,4202503,Bom Jardim da Serra,SC,-28.126091666666667,-49.476494444444334
1020633,126.1,"COMANDO DA AERONAUTICA | ACC-U C-261/450, MORR...",50415723132,507,1007691252,-28.0157,-49.5925,4218905,Urubici,SC,...,<NA>,Urubici,-28.0157,-49.5925,SC,4202503,Bom Jardim da Serra,SC,-28.126091666666667,-49.476494444444334
1020634,132.05,"COMANDO DA AERONAUTICA | VOLMET B-260/450, MOR...",50415723132,507,1007691252,-28.0157,-49.5925,4218905,Urubici,SC,...,<NA>,Urubici,-28.0157,-49.5925,SC,4202503,Bom Jardim da Serra,SC,-28.126091666666667,-49.476494444444334
1020661,131.225,"GOL LINHAS AEREAS S.A. | AOC U 100/100, MACEIÓ",50415208017,507,1011664914,-9.47783,-35.8394,2707701,Rio Largo,AL,...,<NA>,Rio Largo,-9.47783,-35.8394,AL,2704302,Maceió,AL,-9.515,-35.7930555555555


In [ ]:
g.input_info_from_coords()

  0%|          | 0/161 [00:00<?, ?it/s]

Logging: Requisitando API geocoders para locais fora do perímetro brasileiro.

In [ ]:
g.df.isna().sum()

Frequência                           0
Entidade                         17039
Fistel                            1587
Serviço                           1587
Estação                           5401
Latitude                             0
Longitude                            0
Código_Município                   161
Município                          122
UF                                 136
Classe                            2032
Classe_Emissão                    7029
Largura_Emissão(kHz)              2008
Validade_RF                       9240
Status                            1587
Fonte                                0
Multiplicidade                       0
Log                                  0
Cota_Base_Torre(m)             1010064
Potência_Transmissor(W)         995564
Ganho_Antena(dBd)               996917
Ângulo_Elevação_Antena          996566
Azimute_Antena                  995932
Altura_Antena(m)                995146
Atenuação_Linha(db/100m)        996753
Perdas_Acessórias_Linha(d